#### [Pytorch 기반 회귀 모델 구현]   
- Layer => Full-Connected Layer, Linear    
- 손실함수 => MSELoss, MAELoss    

In [39]:
import pandas as pd

df=pd.read_csv("./BostonHousing.csv")
feature = df[df.columns[:-1]]
target = df[df.columns[-1]]

In [40]:
# 머신러닝으로 만들기 
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(feature, target)

# 모델 평가
from sklearn.metrics import mean_squared_error
prediction = model.predict(feature)
mse = mean_squared_error(target, prediction)
print("MSE: ", mse)

MSE:  21.894831181729206


In [41]:
# 딥러닝으로 만들기
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# 데이터셋 클래스 생성
class BostonDataset(Dataset):
    def __init__(self, df):
        super().__init__()
        df=pd.read_csv("./BostonHousing.csv")
        feature = df[df.columns[:-1]]
        target = df[df.columns[-1]]
        self.feature = torch.tensor(feature.values, dtype=torch.float32)
        self.target = torch.tensor(target.values, dtype=torch.float32).view(-1, 1)
    
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self, idx):
        return self.feature[idx], self.target[idx]
    
    def normalize(self):
        self.feature = (self.feature - self.feature.mean()) / self.feature.std()
        self.target = (self.target - self.target.mean()) / self.target.std()

In [42]:
trainDS = BostonDataset(df)

In [43]:
import torch.nn as nn
import torch.nn.functional as F

class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(13, 1)
    
    def forward(self, x):
        return self.linear(x)

In [44]:
import numpy as np

model = LinearModel()
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters())
max=1000
train_loader = DataLoader(trainDS, batch_size=10, shuffle=True)

for epoch in range(1000):
    for feature, target in train_loader:
        optimizer.zero_grad()
        output = model(feature)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if loss < max :
            max=loss
            torch.save(model, "optim.pth")
    if epoch % 100 == 0:
        print("epoch: {}, loss: {}".format(epoch, 0 if np.nan==loss.item() else loss.item()))

epoch: 0, loss: 1551.9984130859375


epoch: 100, loss: 24.422204971313477
epoch: 200, loss: 12.800212860107422
epoch: 300, loss: 61.30430221557617
epoch: 400, loss: 23.849180221557617
epoch: 500, loss: 29.998952865600586
epoch: 600, loss: 119.99533081054688
epoch: 700, loss: 15.300082206726074
epoch: 800, loss: 26.568166732788086
epoch: 900, loss: 14.348891258239746


In [45]:
model=torch.load("optim.pth")
prediction = model(trainDS.feature)
mse = criterion(prediction, trainDS.target)
print("MSE: ", mse.item())

MSE:  26.40672492980957
